# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Project6-2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [ ]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/Project6-2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [ ]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
from typing import Text,Any
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency as em


## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [ ]:
OpenAIClient = em.LoadOpenAIClient(path)

### (2) audio to text

In [ ]:
audio_path = path + 'audio/'
a = em.AudioToText(audio_path, 'audio4.mp3', OpenAIClient)
a

'아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리가 어지럽고 속이 매스꺼워요. 어떻게 해야 할까요?\n'

### (3) text summary

In [ ]:
sumText = em.TextToSummary(a, OpenAIClient)
sumText

'머리를 박아서 3시간 후에 어지러움과 메스꺼움 증상 발생 중'

### (4) 응급실 등급분류

In [ ]:
model_path = path + 'fine_tuned_bert'
#tokenizer = AutoTokenizer.from_pretrained(model_path)
#model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
em.predict(sumText,  model_path, 'cpu')

1

### (5) 응급실추천

In [ ]:
df_emergency = pd.read_csv(path + 'emergency_room.csv')
df_emergency

,Category,Name,Address,Tel,Emergency tel,latitude,longitude
0,응급실운영신고기관,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),052-220-3300,052-220-3334,35.548238,129.307011
1,지역응급의료기관,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,051-723-0171,051-723-2119,35.236029,129.216492
2,지역응급의료기관,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",032-726-1000,032-726-1190,37.508994,126.669479
3,지역응급의료센터,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",031-8021-2114,031-8021-2130,37.234641,127.210499
4,지역응급의료기관,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
522,지역응급의료센터,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",031-467-9717,031-467-9119,37.393404,126.924477
523,지역응급의료센터,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",031-389-3000,031-389-3119,37.358645,126.947360
524,응급실운영신고기관,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),051-709-3000,051-709-3119,35.185413,129.121459
525,응급실운영신고기관,흑룡의원,인천광역시 옹진군 백령면 백령로 831,032-837-6873,032-837-3153,37.959524,124.665499


In [ ]:
df_emergency = df_emergency.rename(columns={'latitude': '위도', 'longitude': '경도'})

In [ ]:
df_emergency

,Category,Name,Address,Tel,Emergency tel,위도,경도
0,응급실운영신고기관,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),052-220-3300,052-220-3334,35.548238,129.307011
1,지역응급의료기관,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,051-723-0171,051-723-2119,35.236029,129.216492
2,지역응급의료기관,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",032-726-1000,032-726-1190,37.508994,126.669479
3,지역응급의료센터,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",031-8021-2114,031-8021-2130,37.234641,127.210499
4,지역응급의료기관,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
522,지역응급의료센터,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",031-467-9717,031-467-9119,37.393404,126.924477
523,지역응급의료센터,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",031-389-3000,031-389-3119,37.358645,126.947360
524,응급실운영신고기관,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),051-709-3000,051-709-3119,35.185413,129.121459
525,응급실운영신고기관,흑룡의원,인천광역시 옹진군 백령면 백령로 831,032-837-6873,032-837-3153,37.959524,124.665499


In [ ]:
em.GetDistance(37.4665, 126.9780, 37.4563, 126.7052)

(34.4, 37.75)

In [ ]:
my_location = (37.339073,127.966817)
em.RecommandHospital(my_location, df_emergency)

,Category,Name,Address,Tel,Emergency tel,위도,경도,거리(km),소요시간(분)
0,지역응급의료기관,의료법인성지의료재단성지병원,강원특별자치도 원주시 원일로 22 (인동),033-760-3101,033-760-3119,37.345299,127.954009,2.21,7.09
1,지역응급의료기관,강원특별자치도원주의료원,"강원특별자치도 원주시 서원대로 387 (개운동, 지방공사강원도원주의료원)",033-761-6911,033-760-4701,37.332496,127.948962,2.77,6.82
2,권역응급의료센터,연세대학교원주세브란스기독병원,강원특별자치도 원주시 일산로 20 (일산동),033-741-0114,033-741-1641,37.347955,127.945645,2.92,9.31


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [ ]:
def pipeline(path, audio, location):
  OpenAIClient = em.LoadOpenAIClient(path)
  audio_path = path + 'audio/'
  audioName = audio + '.mp3'
  a = em.AudioToText(audio_path, audioName, OpenAIClient)
  sumText = em.TextToSummary(a, OpenAIClient)
  model_path = path + 'fine_tuned_bert'
  result = em.predict(sumText,  model_path, 'cpu')
  if(result < 3):
    print('응급상황인것으로 판단됩니다. 응급실로 이동하세요')
  else:
    print('응급상황이 아닌 것으로 판단됩니다. 증상이 악화된다면 응급실로 이동하세요')
  return em.RecommandHospital(location, df_emergency)


In [ ]:
pipeline(path, 'audio2', (38.339073,127.966817))

응급상황이 아닌 것으로 판단됩니다. 증상이 악화된다면 응급실로 이동하세요


,Category,Name,Address,Tel,Emergency tel,위도,경도,거리(km),소요시간(분)
0,지역응급의료기관,의료법인성심의료재단양구성심병원,강원특별자치도 양구군 양구읍 중심로 160,033-480-8803,033-480-8816,38.104779,127.991197,20.18,26.54
1,지역응급의료기관,화천군보건의료원,강원특별자치도 화천군 화천읍 강변로 111 (화천군보건의료원),033-441-4000,033-440-2880,38.107382,127.708883,62.07,67.99
2,권역응급의료센터,한림대학교춘천성심병원,강원특별자치도 춘천시 삭주로 77 (교동),033-240-5000,033-240-5219,37.884136,127.739895,62.11,66.78
